## What do we have to make 

user will give you a situation. 
first agent will ask the additional questions if necessary 

second and third agent will provide pros and cons of the situation. If it requires any tool calls have a web search and simple calculator tool powered by python make both tools availalbe to this agent. 

forth agent: 
will provide the final conclusion if user should take that decision or not. 

## Imports

In [5]:
from dotenv import load_dotenv
load_dotenv()

from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
import asyncio
import nest_asyncio
from duckduckgo_search import DDGS


nest_asyncio.apply()




### Tools for internet search 

In [50]:
from pydantic_ai.tools import Tool

def search_internet(query: str) -> list[dict]: 
    """Searches the web for given query and returns top 5 results

    Args:
        query (str): Query to search

    Returns:
        list: List of top 5 results
    """
    results = DDGS().text(query, max_results=5)
    return results

def get_more_info_from_user(questions: list[str]) -> str:
    """
    In case of more infromation needed, this function asks user relavent info and gets you that info. 

    Args:
        questions (list[str]): list of questions to ask 

    Returns:
        str: answers for above questions from user
    """
    print("------------------------")
    print("Please answer following questions")
    print(questions)
    print("------------------------")
    answers = input()
    return answers

search_tool  = Tool(search_internet)
get_more_info_from_user = Tool(get_more_info_from_user)


In [51]:

model = OpenAIModel('gpt-4o-mini')
entry_and_info_gathering = Agent(  
    system_prompt='You are a expert in making decisions you make right decisions always. You will be given a situation and you have to ask questions to user to so that you can make a right decision which user asks. ',  
    model = model, 
    tools = [search_tool, ask_more_info_from_user]
)

pros_identifier_agent  = Agent(
    system_prompt = "you have given a situation and some questions answered by user. You have to respond all the pros of that situation. ",
    result_type=list[str], 
    model = model , 
    tools = [search_tool]
)

cons_identifier_agent = Agent(
    system_prompt = "you have given a situation and some questions answered by user. You have to respond all the cons of that situation. ",
    result_type=list[str], 
    model = model,
    tools = [search_tool]
)

conclusion_agent = Agent(
    system_prompt = "you have given a situation and some questions answered by user. Also the pros and cons about the situation you have provide conclusion for making the better decision. The highest pros and a decision which is ethical is usually the best decision, think and answer based on the knowledge and scenarios you've observed from past.",
    result_type=str,
    model = model,
    tools = [search_tool]
)

In [53]:
async def main():
    async with pros_identifier_agent.run_stream(
        "I'm going through breakup what to do.", 
    ) as result:  
        print(result)
        async for message in result.stream_text():  
            print(message)

asyncio.run(main())

StreamedRunResult(_all_messages=[ModelRequest(parts=[SystemPromptPart(content='you have given a situation and some questions answered by user. You have to respond all the pros of that situation. ', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="I'm going through breakup what to do.", timestamp=datetime.datetime(2025, 2, 9, 9, 37, 55, 715939, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[ToolCallPart(tool_name='get_more_info_from_user', args='{"questions":["How long was the relationship?","How did the breakup happen?","What are your current feelings about the breakup?","Do you have a support system (friends, family) in place?","What are you looking to achieve in moving on from this breakup?"]}', tool_call_id='call_k9goltM6ONgxicilXssMsTHF', part_kind='tool-call')], model_name='gpt-4o-mini', timestamp=datetime.datetime(2025, 2, 9, 9, 37, 56, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolRe

UserError: stream_text() can only be used with text responses

In [2]:
# %% [markdown]
# ## What do we have to make 
# 
# user will give you a situation. 
# first agent will ask the additional questions if necessary 
# 
# second and third agent will provide pros and cons of the situation. If it requires any tool calls have a web search and simple calculator tool powered by python make both tools availalbe to this agent. 
# 
# forth agent: 
# will provide the final conclusion if user should take that decision or not. 

# %% [markdown]
# ## Imports

# %%
from dotenv import load_dotenv
load_dotenv()

from openai import BaseModel
from rich import print
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
import asyncio
import logfire
from dataclasses import dataclass


logfire.configure()
import nest_asyncio
nest_asyncio.apply()
from duckduckgo_search import DDGS
from rich.prompt import Prompt




# %% [markdown]
# ### Tools for internet search 

# %%
from pydantic_ai.tools import Tool

def search_internet(query: str) -> list[dict]: 
    """Searches the web for given query and returns top 5 results

    Args:
        query (str): Query to search

    Returns:
        list: List of top 5 results
    """
    results = DDGS().text(query, max_results=5)
    return results

async def get_more_info_from_user(questions: list[str]) -> str:
    """
    In case of more infromation needed, this function asks user relavent info and gets you that info. 

    Args:
        questions (list[str]): list of questions to ask 

    Returns:
        str: answers for above questions from user
    """
    answers = Prompt.ask(f"""
    -----
    Please answer the following questions
    -----
    {questions}
    -----
    """)
    return answers

search_tool  = Tool(search_internet)
get_more_info_from_user = Tool(get_more_info_from_user)


# %%
class InfoGatheringOutput(BaseModel): 
    """The model returned by info gathering agent.

    Args:
        BaseModel (_type_): _description_

    Returns:
        _type_: _description_
    """
    query: str 
    questions: list[str]
    answers_from_user: str 

class ProsOutput(BaseModel): 
    """
    The model returned by pros agent.
    Args:
        BaseModel (_type_): _description_

    Returns:
        _type_: _description_
    """
    pros: list[str]

class ConsOutput(BaseModel): 
    """
    The model returned by cons agent.

    Args:
        BaseModel (_type_): _description_

    Returns:
        _type_: _description_
    """
    cons: list[str]

@dataclass
class Deps:
    pros_identifier_agent: Agent
    cons_identifier_agent: Agent
    conclusion_agent: Agent


model = OpenAIModel('gpt-4o-mini')
entry_and_info_gathering = Agent(  
    system_prompt='You are a expert in making decisions you make right decisions always. You will be given a situation and you have to ask questions to user to so that you can make a right decision which user asks. You have access to the tool call for the getting more info from user please use that, after gathering you can use pros and cons gathering agent to get pros and cons. ',  
    model = model, 
    tools = [search_tool, get_more_info_from_user],
    result_type=InfoGatheringOutput
)


pros_identifier_agent  = Agent(
    system_prompt = "you have given a situation and some questions answered by user. You have to respond all the pros of that situation. ",
    result_type=ProsOutput, 
    model = model, 
    tools = [search_tool]
)

cons_identifier_agent = Agent(
    system_prompt = "You are the best critic which evaluates situation and gives critical responses which are importatnt to make decision. you have given a situation and some questions answered by user. You have to respond all the cons of that situation. ",
    result_type=ConsOutput, 
    model = model,
    tools = [search_tool]
)

conclusion_agent = Agent(
    system_prompt = "you have given a situation and some questions answered by user. Also the pros and cons about the situation you have provide conclusion for making the better decision. The highest pros and a decision which is ethical is usually the best decision, think and answer based on the knowledge and scenarios you've observed from past.",
    result_type=str,
    model = model,
    tools = [search_tool]
)

@entry_and_info_gathering.tool
def transfer_to_pros_agent(ctx: RunContext, query: str, questions: list[str], answer: str ) -> str:
    print(f"Getting pros for: {query}")
    result = ctx.deps.pros_identifier_agent.run_sync(f"Please evaluate the topic and provide pros for this {query} previous agent asked this question to the user {questions} and user provided following answer {answer}.")
    return result

@entry_and_info_gathering.tool
def transfer_to_cons_agent(ctx: RunContext, query: str, questions: list[str], answer: str ) -> str:
    print(f"Getting cons for: {query}")
    result = ctx.deps.cons_identifier_agent.run_sync(f"Please evaluate the topic and provide cons for this {query} previous agent asked this question to the user {questions} and user provided following answer {answer}.")
    return result

@pros_identifier_agent.tool
@cons_identifier_agent.tool
def transfer_to_conclusion_agent(ctx: RunContext, query: str, pros: list[str], cons: list[str] ) -> str:
    print(f"Getting conclusion for: {query}")
    result = ctx.deps.conclusion_agent.run_sync(f"Evalute the pros {pros} and cons{cons} should we do this {query}")
    return result 

deps = Deps(pros_identifier_agent = pros_identifier_agent, cons_identifier_agent = cons_identifier_agent, conclusion_agent = conclusion_agent)

# %%
async def main(query: str):
    res =await entry_and_info_gathering.run(query, deps = deps)
    print(res.data)
    return res 

if __name__ == "__main__":
    asyncio.run(main(query = "I want to create AI project which people will use?."))

# %%





13:50:46.059 entry_and_info_gathering run prompt=I want to create AI project which people will use?.
13:50:46.081   preparing model request params run_step=1
13:50:46.082   model request


Logfire project URL: ]8;id=468619;https://logfire.pydantic.dev/somesh/decision-maker\https://logfire.pydantic.dev/somesh/decision-maker]8;;\

13:50:49.251   handle model response
13:50:49.252     running tools=['get_more_info_from_user']


-----
    Please answer the following questions
    -----
    ['What specific problem do you want your AI project to solve?', 'Who is your target audience or user base for 
this project?', 'What type of AI technology are you considering (e.g., machine learning, natural language 
processing, computer vision)?', 'What is your budget for this project?', 'Do you have any existing skills or 
resources that will help you in this project?']
    -----
    :

13:51:04.254   preparing model request params run_step=2
13:51:04.255   model request
13:51:08.699   handle model response
13:51:08.699     running tools=['transfer_to_pros_agent', 'transfer_to_cons_agent']


Getting pros for: AI project for the general public using Python and transformers

13:51:08.707     pros_identifier_agent run prompt=Please evaluate the topic and provide pros for this AI project...answer AGI, for all, Python, transformers, a billion dollars..

Getting cons for: AI project for the general public using Python and transformers


13:51:08.708       preparing model request params run_step=1
13:51:08.709       model request
13:51:08.715     cons_identifier_agent run prompt=Please evaluate the topic and provide cons for this AI project...answer AGI, for all, Python, transformers, a billion dollars..
13:51:08.715       preparing model request params run_step=1
13:51:08.716       model request
                 pros_identifier_agent run prompt=Please evaluate the topic and provide pros for this AI project...answer AGI, for all, Python, transformers, a billion dollars..
13:51:12.782       handle model response
                 cons_identifier_agent run prompt=Please evaluate the topic and provide cons for this AI project...answer AGI, for all, Python, transformers, a billion dollars..
13:51:22.946       handle model response
13:51:22.958   preparing model request params run_step=3
13:51:22.958   model request
13:51:27.448   handle model response


InfoGatheringOutput(
    query='AI project for the general public using Python and transformers',
    questions=[
        'What potential benefits can this project offer to users?',
        'How can it improve their daily lives?',
        'What market demand exists for this type of AI application?',
        'What are the potential downsides or challenges of this project?',
        'What risks are associated with developing AGI?',
        'How competitive is the market for similar AI projects?'
    ],
    answers_from_user='AGI, for all, Python, transformers, a billion dollars.',
    pros=[
        'Accessibility of AGI tools for everyone',
        'Potential for significant cost savings in various sectors',
        'Enhancement of productivity and efficiency in daily tasks',
        'Facilitation of learning and education through AI assistance',
        'Opportunities for innovation in technology and business'
    ],
    cons=[
        'Development of AGI raises ethical concerns regarding control and alignment with human values.',
        'High financial investment of a billion dollars increases the pressure for success, which may lead to 
compromised safety and quality.',
        'Technical challenges in developing robust and reliable AGI could lead to significant delays and increased 
costs.',
        'Risk of bias and discrimination in AI models if data used for training is not carefully managed.',
        'Market competition may lead to rushed development, resulting in incomplete or flawed systems being 
deployed to the public.'
    ]
)